In [82]:
import os
import numpy as np
import matplotlib.pyplot as plt
x_ai_path = (os.path.abspath(os.path.join("", os.pardir)))
intra_day_csv = os.path.join(x_ai_path, "Data","Data","Smard","IntraDay","IntraDayPreisindexIdAEPvonNetztransparenzDE.csv")
allData_csv = os.path.join(x_ai_path, "submission", "merged_data", "allData.csv")

import pandas as pd
df_all_data = pd.read_csv(allData_csv)
df_intra_day = pd.read_csv(intra_day_csv, delimiter=';', decimal=',')

Leider enthält die CSV der Intraday Daten wesentlich genauere Werte als unsere andere CSV, die nur ganze Stunden enthält. Außerdem enthält sie aktuell Werte über einen längeren Zeitraum, die beschnitten werden müssen. 

In [ ]:
df_intra_day['(Uhrzeit) von'] = pd.to_datetime(df_intra_day['Datum von'] + ' ' + df_intra_day['(Uhrzeit) von'], format='%Y-%m-%d %H:%M')
df_intra_day_filtered = df_intra_day[df_intra_day['(Uhrzeit) von'].dt.minute == 0]
earliest_datetime = pd.Timestamp("2020-06-30 22:00:00")
cutoff_datetime = pd.Timestamp("2024-12-23 10:00:00")
df_intra_day_filtered = df_intra_day_filtered[df_intra_day_filtered['(Uhrzeit) von'] <= cutoff_datetime]
print(df_intra_day_filtered)

Nun muss auch noch die große CSV gefiltert werden, um sich auf den gleichen Zeitraum zu beschränken

In [ ]:
df_all_data['Date'] = pd.to_datetime(df_all_data['Date'], format='%Y-%m-%d %H:%M:%S')
earliest_datetime = pd.Timestamp("2020-06-30 22:00:00")
df_all_data_filtered = df_all_data[
    (df_all_data['Date'] >= earliest_datetime) & (df_all_data['Date'] <= cutoff_datetime)
]
print(df_all_data_filtered)

Now we do a cross correlation analysis

In [ ]:
series1 = df_all_data_filtered['day_ahead_prices_EURO_x'].fillna(0)
series2 = df_intra_day_filtered['ID AEP in €/MWh'].fillna(0)

cross_corr_value = np.corrcoef(series1, series2)[0, 1]
print(cross_corr_value)